## これは何?

初心者向け講座#1の内容から学習・予測に不要な部分を排除した notebook です。メインの処理を追い駆けたい!という時にお使いください。

## Note:

**事前学習済みモデルは利用禁止です!**

### 基本の設定

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

import pandas as pd
import numpy as np
from glob import  glob

import matplotlib.pyplot as plt
import seaborn as sns

### 各種ディレクトリの定義

In [3]:
dataset_root = '/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11'
assert dataset_root is not None

input_dir = os.path.join(dataset_root, "inputs")
photo_dir = os.path.join(input_dir, "photos")

output_dir = os.path.join(dataset_root, "output_ver5")
os.makedirs(output_dir, exist_ok=True)

train_df = pd.read_csv(os.path.join(input_dir, 'train.csv'))
test_df = pd.read_csv(os.path.join(input_dir, 'test.csv'))

material_df = pd.read_csv(os.path.join(input_dir, 'materials.csv'))
technique_df = pd.read_csv(os.path.join(input_dir, 'techniques.csv'))

In [4]:
using_models = [os.path.join(output_dir, "0_591_0.7714605069966805.pth"), os.path.join(output_dir, "1_534_0.7553703604594386.pth"), os.path.join(output_dir, "2_576_0.7535785985808189.pth"), os.path.join(output_dir, "3_581_0.7311586328898874.pth"), os.path.join(output_dir, "4_558_0.7744370364462523.pth")]
N_TTA = 100

In [5]:
class Config:
    N_FOLDS = 5
    N_EPOCHS = 30

### 画像データの読み込み

In [6]:
from PIL import Image

def to_img_path(object_id):
    return os.path.join(photo_dir, f'{object_id}.jpg')

def read_image(object_id):
    return Image.open(to_img_path(object_id))

In [7]:
!pip uninstall scikit-learn
!pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn

Found existing installation: scikit-learn 1.0.dev0
Uninstalling scikit-learn-1.0.dev0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.dev0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.0.dev0
Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/scipy-wheels-nightly/simple
     |████████████████████████████████| 22.9 MB 191 kB/s 


In [8]:
import torch
from torch import nn
from torch.optim import Adam
from torch.optim.optimizer import Optimizer
from torch.utils import data

# torchvision
from torchvision import transforms as T
from torchvision.models import resnet34

# scikit-learn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedGroupKFold

In [9]:
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

class AtmaDataset(data.Dataset):
    """atmaCup用にデータ読み込み等を行なうデータ・セット"""
    object_path_key = "object_path"
    label_key = "target"

    @property
    def meta_keys(self):
        retval = [self.object_path_key]

        if self.is_train:
            retval += [self.label_key]

        return retval

    def __init__(self, meta_df: pd.DataFrame, is_train=True):
        """
        args:
            meta_df: 
                画像へのパスと label 情報が含まれている dataframe
                必ず object_path に画像へのパス, target に正解ラベルが入っている必要があります
            
            is_train:
                True のとき学習用のデータ拡張を適用します.
                False の時は単に size にリサイズを行います
        """

        self.is_train = is_train
        for k in self.meta_keys:
            if k not in meta_df:
                raise ValueError("meta df must have {}".format(k))

        self.meta_df = meta_df.reset_index(drop=True)
        self.index_to_data = self.meta_df.to_dict(orient="index")

        size = (256, 256)

        additional_items = (
            [T.Resize(size)]
            if not is_train
            else [
                T.RandomGrayscale(p=0.2),
                T.RandomVerticalFlip(),
                T.RandomHorizontalFlip(),
                T.ColorJitter(
                    brightness=0.3,
                    contrast=0.5,
                    saturation=[0.8, 1.3],
                    hue=[-0.05, 0.05],
                ),
                T.RandomResizedCrop(size),
            ]
        )

        self.transformer = T.Compose(
            [*additional_items, T.ToTensor(), T.Normalize(mean=IMG_MEAN, std=IMG_STD)]
        )

    def __getitem__(self, index):
        data = self.index_to_data[index]

        obj_path, label = data.get(self.object_path_key), data.get(self.label_key, -1)
        img = Image.open(obj_path)
        img = self.transformer(img)
        return img, label

    def __len__(self):
        return len(self.meta_df)

In [10]:
# CUDA を使うので確認. google colab の場合 GPU accelerator をオンにしておいてください
assert torch.cuda.is_available()

DEVICE = torch.device("cuda")

## Train / Validation Phase

In [11]:
def train(
    model: nn.Module,
    optimizer: Optimizer,
    train_loader: data.DataLoader
) -> pd.Series:

    # train にすることで model 内の学習時にのみ有効な機構が有効になります (Dropouts Layers、BatchNorm Layers...)
    model.train()
    
    criterion = nn.MSELoss()
    
    for i, (x_i, y_i) in enumerate(train_loader):
        x_i = x_i.to(DEVICE)
        y_i = y_i.to(DEVICE).reshape(-1, 1).float()

        output = model(x_i)
        loss = criterion(output, y_i)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def predict(model: nn.Module, loader: data.DataLoader) -> np.ndarray:
    # train とは逆で model 内の学習時にのみ有効な機構がオフになります (Dropouts Layers、BatchNorm Layers...)
    model.eval()
    
    predicts = []
    
    for x_i, y_i in loader:
        
        # 明示的に勾配を計算しないように指定することができます. 
        # この関数ではモデルの更新はせずに単に出力だけを使いますので勾配は不要です.
        with torch.no_grad():
            output = model(x_i.to(DEVICE))

        predicts.extend(output.data.cpu().numpy())

    pred = np.array(predicts).reshape(-1)
    return pred


def calculate_metrics(y_true, y_pred) -> dict:
    """正解ラベルと予測ラベルから指標を計算する"""    
    return {
        'rmse': mean_squared_error(y_true, y_pred) ** .5
    }


def valid(
    model: nn.Module, 
    y_valid: np.ndarray, 
    valid_loader: data.DataLoader
) -> pd.Series:
    """検証フェーズ
    与えられたモデル・データローダを使って検証フェーズを実行。スコアの dict と予測した値を返す
    """
    
    pred = predict(model, valid_loader)
    score = calculate_metrics(y_valid, pred)
    return score, pred

## Run Fold

1. train / valid の loader 作成
2. 以下を epoch 数だけ繰り返す
    1. 学習用データで学習 
    2. 検証用データで検証スコアの算出

In [12]:
# def calc_cv(
#     model: nn.Module, 
#     valid_df: pd.DataFrame, 
#     y_valid: np.ndarray,
#     n_tta: int) -> float:
#     """
#     train / valid に分割されたデータで学習と同時に検証を行なう
#     """
    
#     #   : 検証用の方は is_train=False にしてデータ拡張オフにする
#     valid_dataset = AtmaDataset(meta_df=valid_df, is_train=False)
#     valid_loader = data.DataLoader(valid_dataset, batch_size=256, num_workers=4)
    
#     # optimizer の定義
#     optimizer = Adam(model.parameters(), lr=1e-3)

#     best_score = float('inf')
#     best_model_path = None
#     best_model = None

#     model_scores = []

#     for epoch in range(1, n_epochs + 1):
#         print(f'start {epoch}')
        
#         # 1: 学習用データで学習を実行。学習時のロスを取得
#         train(model, optimizer, train_loader)

#         # 2: 検証データでのスコアを計算
#         score_valid, y_valid_pred = valid(model=model, valid_loader=valid_loader, y_valid=y_valid)

#         model_scores.append(score_valid['rmse'])

#         model_path = os.path.join(output_dir, str(i) + '_' + str(epoch) + '_' + str(score_valid['rmse']) + '.pth')

#         if best_score > score_valid['rmse']:
#             best_score = score_valid['rmse']
#             best_model_path = model_path
#             best_model = model.state_dict()
    
#     torch.save(best_model, best_model_path)

#     torch.save(model.state_dict(), model_path)

#     fig = plt.figure()

#     plt.plot(list(range(n_epochs)), model_scores)

#     fig.savefig(os.path.join(output_dir, "scores_(" + str(i) + ")_" + str(1) + "-" + str(n_epochs) + ").png"))

#     return best_score, best_model_path

### その他

モデル作成などの関数定義

In [13]:
def create_model():
    model = resnet34(pretrained=False)
    model.fc = nn.Linear(in_features=512, out_features=1, bias=True)    
    return model
    
def create_metadata(input_df):
    out_df = input_df[['object_id']].copy()
    out_df['object_path'] = input_df['object_id'].map(to_img_path)
    
    if "target" in input_df:
        out_df["target"] = (input_df['sorting_date'] - 1550) / 100

    return out_df

# def run_test_predict(model):
#     test_meta_df = create_metadata(test_df)

#     # 学習時のデータ拡張はオフにしたいので is_train=False としている
#     test_dataset = AtmaDataset(meta_df=test_meta_df, is_train=False)
#     test_loader = data.DataLoader(dataset=test_dataset, batch_size=128, drop_last=False, num_workers=4)
    
#     y_pred = predict(model, loader=test_loader)
#     return y_pred

In [14]:
from tqdm import tqdm

def run_test_predict(model, input_df, n_tta=0):
    # n_tta > 0 の時だけデータ拡張を on にする (is_train = True)
    is_tta_mode = n_tta > 0
    test_dataset = AtmaDataset(meta_df=input_df, is_train=is_tta_mode)
    test_loader = data.DataLoader(dataset=test_dataset, batch_size=128, drop_last=False, num_workers=4)

    predictions = []
    n_times = 1 if not is_tta_mode else n_tta
    print(f"run #{n_times} times / tta={is_tta_mode}")
    for _ in tqdm(range(n_times)):
        y_pred = predict(model, loader=test_loader)
        predictions.append(y_pred)

    return np.array(predictions).mean(axis=0)

In [15]:
train_df.columns

Index(['object_id', 'sorting_date', 'art_series_id', 'target'], dtype='object')

In [16]:
def total_rmse(model_scores):
    total_rmse = 0
    for model_score in model_scores:
      total_rmse += model_score ** 2
    total_rmse /= 5
    total_rmse = total_rmse ** 0.5
    print('total_rmse: ' + str(total_rmse))

In [17]:
train_meta_df = create_metadata(train_df)

test_predictions = []

fold = StratifiedGroupKFold(n_splits=5, shuffle=False)
cv = list(fold.split(X=train_df, y=train_df['target'], groups=train_df['art_series_id']))[:Config.N_FOLDS]

model_scores = []

for i, (idx_tr, idx_valid) in enumerate(cv):
    model = create_model()
    model.to(DEVICE)
    model_path = using_models[i]
    print(model_path)
    model.load_state_dict(torch.load(model_path))

    valid_meta_df=train_meta_df.iloc[idx_valid]
    y_valid=train_meta_df['target'].values[idx_valid]
    
    y_pred_tta = run_test_predict(model, valid_meta_df, n_tta=N_TTA)
    model_score = calculate_metrics(y_valid, y_pred_tta)

    model_scores.append(model_score['rmse'])

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_ver5/0_591_0.7714605069966805.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/100 [00:00<?, ?it/s]

run #100 times / tta=True


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 100/100 [11:27<00:00,  6.87s/it]


/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_ver5/1_534_0.7553703604594386.pth


...

100%|██████████| 100/100 [11:28<00:00,  6.89s/it]


/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_ver5/4_558_0.7744370364462523.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/100 [00:00<?, ?it/s]

run #100 times / tta=True


100%|██████████| 100/100 [11:38<00:00,  6.98s/it]


In [18]:
model_scores

[0.7750781124913851,
 0.7503958281591349,
 0.7671288981872134,
 0.7854200514753702,
 0.7407473346350728]

In [19]:
total_rmse(model_scores)

total_rmse: 0.7639264358358477


In [ ]:
test_meta_df = create_metadata(test_df)
test_meta_df['target'] = np.nan

for i in range(5):
    model = create_model()
    model.to(DEVICE)
    model_path = using_models[i]
    print(model_path)
    model.load_state_dict(torch.load(model_path))
    
    # 2. モデルで予測 (本当はローカルに保存した重みを読みだすなどするほうがあとで振り返りやすいが簡易にそのまま予測する)
    y_pred_i = run_test_predict(model, test_meta_df, n_tta=N_TTA)
    test_predictions.append(y_pred_i)
    del model

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_ver6/0_421_0.7383048741578091.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))

  0%|          | 0/100 [00:00<?, ?it/s]

run #100 times / tta=True



100%|██████████| 100/100 [33:47<00:00, 20.27s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_ver6/1_452_0.7136241568659565.pth
run #100 times / tta=True


...


100%|██████████| 100/100 [33:56<00:00, 20.36s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_ver6/3_478_0.7419511815171114.pth
run #100 times / tta=True



100%|██████████| 100/100 [33:41<00:00, 20.22s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_ver6/4_498_0.7419037658906035.pth
run #100 times / tta=True



100%|██████████| 100/100 [33:19<00:00, 20.00s/it]


In [ ]:
# すべての予測の平均値を使う
pred_mean = np.array(test_predictions).mean(axis=0)

pd.DataFrame({
    "target": pred_mean
}).to_csv(os.path.join(output_dir, "submission20210715_1654.csv"), index=False)